In [1]:
# %%
import os
from pathlib import Path
import itertools
import random

import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, utils as vutils

import matplotlib.pyplot as plt
import imageio.v2 as imageio

# ---- Device: CUDA -> MPS (Apple Silicon) -> CPU ----
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
else:
    DEVICE = torch.device("cpu")

print("Using device:", DEVICE)

# ---- Paths (TUKAJ POPRAVI, če imaš dataset drugje) ----
# Ta pot mora kazati na folder, ki vsebuje trainA/, trainB/, testA/, testB/
DATA_ROOT = Path("./apple2orange")  # npr: Path("/Users/viktor/Downloads/apple2orange")

OUT_DIR = Path("./v3_2_outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# ---- Hyperparametri ----
IMG_SIZE = 256
BATCH_SIZE = 1        # CycleGAN klasika: batch_size = 1
NUM_EPOCHS = 15       # minimalno po navodilih (lahko daš več)
LR = 2e-4

LAMBDA_CYCLE = 10.0   # weight za cycle loss
LAMBDA_ID = 5.0       # weight za identity loss (ponavadi 0.5 * LAMBDA_CYCLE)


Using device: mps


In [2]:
# Transformacije (train + test)
transform_train = transforms.Compose([
    transforms.Resize(int(IMG_SIZE * 1.12)),
    transforms.RandomCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5)),
])

transform_test = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5)),
])

class SingleFolderDataset(torch.utils.data.Dataset):
    def __init__(self, folder, transform=None):
        self.folder = Path(folder)
        self.transform = transform
        self.paths = list(self.folder.glob("*.*"))
        self.paths = [p for p in self.paths
                      if p.suffix.lower() in [".jpg", ".jpeg", ".png"]]

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        img = Image.open(path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img

trainA_dir = DATA_ROOT / "trainA"  # jabolka (X)
trainB_dir = DATA_ROOT / "trainB"  # pomaranče (Y)
testA_dir  = DATA_ROOT / "testA"
testB_dir  = DATA_ROOT / "testB"

trainA_ds = SingleFolderDataset(trainA_dir, transform_train)
trainB_ds = SingleFolderDataset(trainB_dir, transform_train)
testA_ds  = SingleFolderDataset(testA_dir,  transform_test)
testB_ds  = SingleFolderDataset(testB_dir,  transform_test)

trainA_loader = DataLoader(trainA_ds, batch_size=BATCH_SIZE, shuffle=True,
                           num_workers=2, pin_memory=True)
trainB_loader = DataLoader(trainB_ds, batch_size=BATCH_SIZE, shuffle=True,
                           num_workers=2, pin_memory=True)

testA_loader = DataLoader(testA_ds, batch_size=1, shuffle=True)
testB_loader = DataLoader(testB_ds, batch_size=1, shuffle=True)

print(f"TrainA (apples):   {len(trainA_ds)} images")
print(f"TrainB (oranges):  {len(trainB_ds)} images")
print(f"TestA (apples):    {len(testA_ds)} images")
print(f"TestB (oranges):   {len(testB_ds)} images")


TrainA (apples):   995 images
TrainB (oranges):  1019 images
TestA (apples):    266 images
TestB (oranges):   248 images


In [3]:
# %% [markdown]
# Dataset apple2orange – pričakovana struktura:
# 
# DATA_ROOT/
#   trainA/  (jabolka)
#   trainB/  (pomaranče)
#   testA/
#   testB/

# %%
from PIL import Image

# Transformacije (train + test)
transform_train = transforms.Compose([
    transforms.Resize(int(IMG_SIZE * 1.12)),
    transforms.RandomCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5)),
])

transform_test = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5)),
])


class SingleFolderDataset(torch.utils.data.Dataset):
    """
    Dataset, ki vzame vse .jpg/.jpeg/.png slike iz ene mape.
    """
    def __init__(self, folder, transform=None):
        self.folder = Path(folder)
        self.transform = transform
        if self.folder.exists():
            self.paths = [p for p in self.folder.glob("*.*")
                          if p.suffix.lower() in [".jpg", ".jpeg", ".png"]]
        else:
            self.paths = []

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        img = Image.open(path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img


# Mape
trainA_dir = DATA_ROOT / "trainA"  # jabolka (X)
trainB_dir = DATA_ROOT / "trainB"  # pomaranče (Y)
testA_dir  = DATA_ROOT / "testA"
testB_dir  = DATA_ROOT / "testB"

print("DATA_ROOT:", DATA_ROOT.resolve())
print("  trainA_dir:", trainA_dir, "exists:", trainA_dir.exists())
print("  trainB_dir:", trainB_dir, "exists:", trainB_dir.exists())
print("  testA_dir :", testA_dir,  "exists:", testA_dir.exists())
print("  testB_dir :", testB_dir,  "exists:", testB_dir.exists())

# Datasets
trainA_ds = SingleFolderDataset(trainA_dir, transform_train)
trainB_ds = SingleFolderDataset(trainB_dir, transform_train)
testA_ds  = SingleFolderDataset(testA_dir,  transform_test)
testB_ds  = SingleFolderDataset(testB_dir,  transform_test)

print(f"TrainA (apples):   {len(trainA_ds)} images")
print(f"TrainB (oranges):  {len(trainB_ds)} images")
print(f"TestA (apples):    {len(testA_ds)} images")
print(f"TestB (oranges):   {len(testB_ds)} images")

# Če ni podatkov, takoj fail, da veš, da je treba popravit pot/dataset
if len(trainA_ds) == 0 or len(trainB_ds) == 0:
    raise RuntimeError(
        "Dataset je prazen!\n"
        f" -> trainA ima {len(trainA_ds)} slik\n"
        f" -> trainB ima {len(trainB_ds)} slik\n"
        "Preveri DATA_ROOT in strukturo map: DATA_ROOT/trainA/*.jpg, DATA_ROOT/trainB/*.jpg"
    )

# DataLoaderji (num_workers=0 = varno na macOS)
trainA_loader = DataLoader(trainA_ds, batch_size=BATCH_SIZE, shuffle=True,
                           num_workers=0, pin_memory=True)
trainB_loader = DataLoader(trainB_ds, batch_size=BATCH_SIZE, shuffle=True,
                           num_workers=0, pin_memory=True)

testA_loader = DataLoader(testA_ds, batch_size=1, shuffle=True) if len(testA_ds) > 0 else None
testB_loader = DataLoader(testB_ds, batch_size=1, shuffle=True) if len(testB_ds) > 0 else None


DATA_ROOT: /Users/viktorrackov/Desktop/Feri/3.semestar/Globoke/V3.2/apple2orange
  trainA_dir: apple2orange/trainA exists: True
  trainB_dir: apple2orange/trainB exists: True
  testA_dir : apple2orange/testA exists: True
  testB_dir : apple2orange/testB exists: True
TrainA (apples):   995 images
TrainB (oranges):  1019 images
TestA (apples):    266 images
TestB (oranges):   248 images


In [4]:
# %% [markdown]
# Dataset apple2orange – pričakovana struktura:
# 
# DATA_ROOT/
#   trainA/  (jabolka)
#   trainB/  (pomaranče)
#   testA/
#   testB/

# %%
from PIL import Image

# Transformacije (train + test)
transform_train = transforms.Compose([
    transforms.Resize(int(IMG_SIZE * 1.12)),
    transforms.RandomCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5)),
])

transform_test = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5)),
])


class SingleFolderDataset(torch.utils.data.Dataset):
    """
    Dataset, ki vzame vse .jpg/.jpeg/.png slike iz ene mape.
    """
    def __init__(self, folder, transform=None):
        self.folder = Path(folder)
        self.transform = transform
        if self.folder.exists():
            self.paths = [p for p in self.folder.glob("*.*")
                          if p.suffix.lower() in [".jpg", ".jpeg", ".png"]]
        else:
            self.paths = []

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        img = Image.open(path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img


# Mape
trainA_dir = DATA_ROOT / "trainA"  # jabolka (X)
trainB_dir = DATA_ROOT / "trainB"  # pomaranče (Y)
testA_dir  = DATA_ROOT / "testA"
testB_dir  = DATA_ROOT / "testB"

print("DATA_ROOT:", DATA_ROOT.resolve())
print("  trainA_dir:", trainA_dir, "exists:", trainA_dir.exists())
print("  trainB_dir:", trainB_dir, "exists:", trainB_dir.exists())
print("  testA_dir :", testA_dir,  "exists:", testA_dir.exists())
print("  testB_dir :", testB_dir,  "exists:", testB_dir.exists())

# Datasets
trainA_ds = SingleFolderDataset(trainA_dir, transform_train)
trainB_ds = SingleFolderDataset(trainB_dir, transform_train)
testA_ds  = SingleFolderDataset(testA_dir,  transform_test)
testB_ds  = SingleFolderDataset(testB_dir,  transform_test)

print(f"TrainA (apples):   {len(trainA_ds)} images")
print(f"TrainB (oranges):  {len(trainB_ds)} images")
print(f"TestA (apples):    {len(testA_ds)} images")
print(f"TestB (oranges):   {len(testB_ds)} images")

# Če ni podatkov, takoj fail, da veš, da je treba popravit pot/dataset
if len(trainA_ds) == 0 or len(trainB_ds) == 0:
    raise RuntimeError(
        "Dataset je prazen!\n"
        f" -> trainA ima {len(trainA_ds)} slik\n"
        f" -> trainB ima {len(trainB_ds)} slik\n"
        "Preveri DATA_ROOT in strukturo map: DATA_ROOT/trainA/*.jpg, DATA_ROOT/trainB/*.jpg"
    )

# DataLoaderji (num_workers=0 = varno na macOS)
trainA_loader = DataLoader(trainA_ds, batch_size=BATCH_SIZE, shuffle=True,
                           num_workers=0, pin_memory=True)
trainB_loader = DataLoader(trainB_ds, batch_size=BATCH_SIZE, shuffle=True,
                           num_workers=0, pin_memory=True)

testA_loader = DataLoader(testA_ds, batch_size=1, shuffle=True) if len(testA_ds) > 0 else None
testB_loader = DataLoader(testB_ds, batch_size=1, shuffle=True) if len(testB_ds) > 0 else None


DATA_ROOT: /Users/viktorrackov/Desktop/Feri/3.semestar/Globoke/V3.2/apple2orange
  trainA_dir: apple2orange/trainA exists: True
  trainB_dir: apple2orange/trainB exists: True
  testA_dir : apple2orange/testA exists: True
  testB_dir : apple2orange/testB exists: True
TrainA (apples):   995 images
TrainB (oranges):  1019 images
TestA (apples):    266 images
TestB (oranges):   248 images


In [5]:
# %% [markdown]
# Modeli (generatorji + diskriminatorji), inicializacija uteži,
# loss funkcije, optimizerji in helper funkcije.

# %%
import torch.nn as nn
import torch.optim as optim
import itertools

# ---- ResNet blok ----
class ResnetBlock(nn.Module):
    """
    ResNet blok: Conv - IN - ReLU - Conv - IN + skip.
    """
    def __init__(self, dim):
        super().__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(dim, dim, kernel_size=3, stride=1, padding=1, bias=False),
            nn.InstanceNorm2d(dim),
            nn.ReLU(True),
            nn.Conv2d(dim, dim, kernel_size=3, stride=1, padding=1, bias=False),
            nn.InstanceNorm2d(dim)
        )

    def forward(self, x):
        return x + self.conv_block(x)

# ---- Generator ----
class ResnetGenerator(nn.Module):
    """
    CycleGAN generator (X -> Y ali Y -> X).
    c7s1-64, d128, d256, R256xN, u128, u64, c7s1-3
    """
    def __init__(self, input_nc=3, output_nc=3, n_filters=64, n_blocks=6):
        super().__init__()

        model = []

        # c7s1-64
        model += [
            nn.Conv2d(input_nc, n_filters, kernel_size=7, stride=1, padding=3, bias=False),
            nn.InstanceNorm2d(n_filters),
            nn.ReLU(True)
        ]

        # downsampling: d128, d256
        in_c = n_filters
        out_c = in_c * 2
        for _ in range(2):
            model += [
                nn.Conv2d(in_c, out_c, kernel_size=3, stride=2, padding=1, bias=False),
                nn.InstanceNorm2d(out_c),
                nn.ReLU(True)
            ]
            in_c = out_c
            out_c *= 2

        # ResNet bloki
        for _ in range(n_blocks):
            model.append(ResnetBlock(in_c))

        # upsampling: u128, u64
        out_c = in_c // 2
        for _ in range(2):
            model += [
                nn.ConvTranspose2d(in_c, out_c, kernel_size=3, stride=2,
                                   padding=1, output_padding=1, bias=False),
                nn.InstanceNorm2d(out_c),
                nn.ReLU(True)
            ]
            in_c = out_c
            out_c //= 2

        # final c7s1-3 + Tanh
        model += [
            nn.Conv2d(in_c, output_nc, kernel_size=7, stride=1, padding=3),
            nn.Tanh()
        ]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)

# ---- PatchGAN diskriminator ----
class PatchDiscriminator(nn.Module):
    """
    PatchGAN diskriminator (70x70).
    """
    def __init__(self, input_nc=3, n_filters=64):
        super().__init__()

        layers = []

        # C64 (brez norm)
        layers += [
            nn.Conv2d(input_nc, n_filters, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, True)
        ]

        # C128, C256, C512
        in_c = n_filters
        out_c = in_c * 2
        for _ in range(3):
            layers += [
                nn.Conv2d(in_c, out_c, kernel_size=4,
                          stride=2 if out_c <= 256 else 1,
                          padding=1, bias=False),
                nn.InstanceNorm2d(out_c),
                nn.LeakyReLU(0.2, True)
            ]
            in_c = out_c
            out_c = min(out_c * 2, 512)

        # izhod: 1 kanal
        layers += [
            nn.Conv2d(in_c, 1, kernel_size=4, stride=1, padding=1)
        ]

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

# ---- Inic. uteži ----
def init_weights(m):
    if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d)):
        nn.init.normal_(m.weight.data, 0.0, 0.02)
        if m.bias is not None:
            nn.init.constant_(m.bias.data, 0.0)
    elif isinstance(m, nn.InstanceNorm2d):
        if m.weight is not None:
            nn.init.normal_(m.weight.data, 1.0, 0.02)
        if m.bias is not None:
            nn.init.constant_(m.bias.data, 0.0)

# ---- Ustvarimo modele ----
G_X2Y = ResnetGenerator(input_nc=3, output_nc=3, n_filters=64, n_blocks=6).to(DEVICE)
G_Y2X = ResnetGenerator(input_nc=3, output_nc=3, n_filters=64, n_blocks=6).to(DEVICE)

D_X = PatchDiscriminator(input_nc=3, n_filters=64).to(DEVICE)  # real vs fake apples
D_Y = PatchDiscriminator(input_nc=3, n_filters=64).to(DEVICE)  # real vs fake oranges

G_X2Y.apply(init_weights)
G_Y2X.apply(init_weights)
D_X.apply(init_weights)
D_Y.apply(init_weights)

print("Models initialized on", DEVICE)

# ---- Loss funkcije ----
criterion_GAN = nn.MSELoss()    # LSGAN
criterion_cycle = nn.L1Loss()   # cycle loss
criterion_identity = nn.L1Loss()# identity loss

# ---- Optimizatorji ----
optimizer_G = optim.Adam(
    itertools.chain(G_X2Y.parameters(), G_Y2X.parameters()),
    lr=LR, betas=(0.5, 0.999)
)
optimizer_D_X = optim.Adam(D_X.parameters(), lr=LR, betas=(0.5, 0.999))
optimizer_D_Y = optim.Adam(D_Y.parameters(), lr=LR, betas=(0.5, 0.999))

# ---- Helperji ----
def set_requires_grad(nets, requires_grad=False):
    """
    Omogoči/onemogoči gradiente za podane mreže.
    """
    if not isinstance(nets, list):
        nets = [nets]
    for net in nets:
        for p in net.parameters():
            p.requires_grad = requires_grad

def denormalize(tensor):
    """
    Pretvori iz [-1, 1] v [0, 1] za prikaz/shranjevanje.
    """
    out = tensor * 0.5 + 0.5
    return torch.clamp(out, 0.0, 1.0)


Models initialized on mps


In [6]:
# %% [markdown]
# Loss funkcije:
# - GAN loss: MSE (LSGAN)
# - Cycle loss: L1
# - Identity loss: L1
# 
# Optimizatorji: Adam za G in oba D.

# %%
# GAN loss (LSGAN)
criterion_GAN = nn.MSELoss()
# Cycle & identity loss
criterion_cycle = nn.L1Loss()
criterion_identity = nn.L1Loss()

# Optimizatorji
optimizer_G = optim.Adam(
    itertools.chain(G_X2Y.parameters(), G_Y2X.parameters()),
    lr=LR, betas=(0.5, 0.999)
)
optimizer_D_X = optim.Adam(D_X.parameters(), lr=LR, betas=(0.5, 0.999))
optimizer_D_Y = optim.Adam(D_Y.parameters(), lr=LR, betas=(0.5, 0.999))

def set_requires_grad(nets, requires_grad=False):
    """
    Omogoči/onemogoči gradiente za podane mreže.
    """
    if not isinstance(nets, list):
        nets = [nets]
    for net in nets:
        for p in net.parameters():
            p.requires_grad = requires_grad

def denormalize(tensor):
    """
    Transformacija iz [-1, 1] v [0, 1] za prikaz/shranjevanje.
    """
    out = tensor * 0.5 + 0.5
    return torch.clamp(out, 0.0, 1.0)


In [9]:
# %% [markdown]
# DEBUG training loop:
# - omejimo število batch-ev na epoh
# - sproti izpisujemo napredek
# Ko preveriš, da dela, lahko omejitev odstraniš.

# %%
G_losses = []
D_X_losses = []
D_Y_losses = []

samples_dir = OUT_DIR / "samples"
samples_dir.mkdir(exist_ok=True, parents=True)

# vzorec za vizualizacijo
sample_X = next(iter(trainA_loader)).to(DEVICE)
sample_Y = next(iter(trainB_loader)).to(DEVICE)

MAX_BATCHES_PER_EPOCH = None  # za debug; kasneje daj na None ali odstrani

for epoch in range(1, NUM_EPOCHS + 1):
    G_running_loss = 0.0
    D_X_running_loss = 0.0
    D_Y_running_loss = 0.0

    # zipamo loaderja (traja do min dolžine)
    for batch_idx, (real_X, real_Y) in enumerate(zip(trainA_loader, trainB_loader), start=1):
        real_X = real_X.to(DEVICE)
        real_Y = real_Y.to(DEVICE)

        # -----------------------
        # 1) Generatorji
        # -----------------------
        set_requires_grad([D_X, D_Y], False)
        optimizer_G.zero_grad()

        # identity
        id_Y = G_X2Y(real_Y)
        loss_id_Y = criterion_identity(id_Y, real_Y)

        id_X = G_Y2X(real_X)
        loss_id_X = criterion_identity(id_X, real_X)

        loss_identity = (loss_id_X + loss_id_Y) * LAMBDA_ID

        # GAN
        fake_Y = G_X2Y(real_X)
        pred_fake_Y = D_Y(fake_Y)
        valid_Y = torch.ones_like(pred_fake_Y, device=DEVICE)
        loss_G_X2Y = criterion_GAN(pred_fake_Y, valid_Y)

        fake_X = G_Y2X(real_Y)
        pred_fake_X = D_X(fake_X)
        valid_X = torch.ones_like(pred_fake_X, device=DEVICE)
        loss_G_Y2X = criterion_GAN(pred_fake_X, valid_X)

        loss_GAN = loss_G_X2Y + loss_G_Y2X

        # cycle
        rec_X = G_Y2X(fake_Y)
        loss_cycle_X = criterion_cycle(rec_X, real_X)

        rec_Y = G_X2Y(fake_X)
        loss_cycle_Y = criterion_cycle(rec_Y, real_Y)

        loss_cycle = (loss_cycle_X + loss_cycle_Y) * LAMBDA_CYCLE

        # total G loss
        loss_G = loss_GAN + loss_cycle + loss_identity
        loss_G.backward()
        optimizer_G.step()

        # -----------------------
        # 2) D_X
        # -----------------------
        set_requires_grad(D_X, True)
        optimizer_D_X.zero_grad()

        pred_real_X = D_X(real_X)
        valid = torch.ones_like(pred_real_X, device=DEVICE)
        loss_D_X_real = criterion_GAN(pred_real_X, valid)

        pred_fake_X = D_X(fake_X.detach())
        fake = torch.zeros_like(pred_fake_X, device=DEVICE)
        loss_D_X_fake = criterion_GAN(pred_fake_X, fake)

        loss_D_X_total = 0.5 * (loss_D_X_real + loss_D_X_fake)
        loss_D_X_total.backward()
        optimizer_D_X.step()

        # -----------------------
        # 3) D_Y
        # -----------------------
        set_requires_grad(D_Y, True)
        optimizer_D_Y.zero_grad()

        pred_real_Y = D_Y(real_Y)
        valid = torch.ones_like(pred_real_Y, device=DEVICE)
        loss_D_Y_real = criterion_GAN(pred_real_Y, valid)

        pred_fake_Y = D_Y(fake_Y.detach())
        fake = torch.zeros_like(pred_fake_Y, device=DEVICE)
        loss_D_Y_fake = criterion_GAN(pred_fake_Y, fake)

        loss_D_Y_total = 0.5 * (loss_D_Y_real + loss_D_Y_fake)
        loss_D_Y_total.backward()
        optimizer_D_Y.step()

        # akumulacija izgub
        G_running_loss += loss_G.item()
        D_X_running_loss += loss_D_X_total.item()
        D_Y_running_loss += loss_D_Y_total.item()

        # DEBUG print
        if batch_idx % 10 == 0:
            print(f"Epoch {epoch}/{NUM_EPOCHS}  Batch {batch_idx}  "
                  f"G: {loss_G.item():.4f}  DX: {loss_D_X_total.item():.4f}  DY: {loss_D_Y_total.item():.4f}")

        # Omejimo št. batch-ev na epoh za debug
        if MAX_BATCHES_PER_EPOCH is not None and batch_idx >= MAX_BATCHES_PER_EPOCH:
            break

    # Povprečja po epohi
    num_batches = min(len(trainA_loader), len(trainB_loader))
    if MAX_BATCHES_PER_EPOCH is not None:
        num_batches = min(num_batches, MAX_BATCHES_PER_EPOCH)

    G_epoch_loss = G_running_loss / num_batches
    D_X_epoch_loss = D_X_running_loss / num_batches
    D_Y_epoch_loss = D_Y_running_loss / num_batches

    G_losses.append(G_epoch_loss)
    D_X_losses.append(D_X_epoch_loss)
    D_Y_losses.append(D_Y_epoch_loss)

    print(f"==> Epoch [{epoch}/{NUM_EPOCHS}]  "
          f"G_loss: {G_epoch_loss:.4f}  "
          f"D_X_loss: {D_X_epoch_loss:.4f}  "
          f"D_Y_loss: {D_Y_epoch_loss:.4f}")

    # -----------------------
    # Vizualizacija po epohi
    # -----------------------
    with torch.no_grad():
        G_X2Y.eval()
        G_Y2X.eval()

        # apple -> orange -> apple
        fake_Y_sample = G_X2Y(sample_X)
        rec_X_sample = G_Y2X(fake_Y_sample)

        # orange -> apple -> orange
        fake_X_sample = G_Y2X(sample_Y)
        rec_Y_sample = G_X2Y(fake_X_sample)

        G_X2Y.train()
        G_Y2X.train()

        sX = denormalize(sample_X.cpu())
        fY = denormalize(fake_Y_sample.cpu())
        rX = denormalize(rec_X_sample.cpu())

        sY = denormalize(sample_Y.cpu())
        fX = denormalize(fake_X_sample.cpu())
        rY = denormalize(rec_Y_sample.cpu())

        def concat3(a, b, c):
            return torch.cat([a, b, c], dim=3)

        grid_X = concat3(sX, fY, rX)
        grid_Y = concat3(sY, fX, rY)
        vis = torch.cat([grid_X, grid_Y], dim=2)

        vis_img = vutils.make_grid(vis, nrow=1)
        vis_img = vis_img.permute(1, 2, 0).numpy()

        epoch_img_path = samples_dir / f"epoch_{epoch:03d}.png"
        plt.imsave(epoch_img_path, vis_img)


/Users/viktorrackov/Desktop/Feri/3.semestar/Globoke/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch 1/15  Batch 10  G: 4.3877  DX: 0.3734  DY: 0.2205
Epoch 1/15  Batch 20  G: 6.4306  DX: 0.2147  DY: 0.3843
Epoch 1/15  Batch 30  G: 12.2935  DX: 0.2412  DY: 0.3707
Epoch 1/15  Batch 40  G: 6.2537  DX: 0.2920  DY: 0.1939
Epoch 1/15  Batch 50  G: 9.7201  DX: 0.3083  DY: 0.3191
Epoch 1/15  Batch 60  G: 7.7409  DX: 0.1982  DY: 0.3684
Epoch 1/15  Batch 70  G: 9.1377  DX: 0.3423  DY: 0.2667
Epoch 1/15  Batch 80  G: 10.0802  DX: 0.1796  DY: 0.1751
Epoch 1/15  Batch 90  G: 7.9856  DX: 0.2260  DY: 0.3025
Epoch 1/15  Batch 100  G: 10.6934  DX: 0.2468  DY: 0.2351
Epoch 1/15  Batch 110  G: 7.8437  DX: 0.3939  DY: 0.1672
Epoch 1/15  Batch 120  G: 5.9666  DX: 0.1889  DY: 0.3992
Epoch 1/15  Batch 130  G: 6.6251  DX: 0.3070  DY: 0.1708
Epoch 1/15  Batch 140  G: 7.7302  DX: 0.2040  DY: 0.1198
Epoch 1/15  Batch 150  G: 7.5809  DX: 0.2122  DY: 0.1753
Epoch 1/15  Batch 160  G: 7.4866  DX: 0.3983  DY: 0.2820
Epoch 1/15  Batch 170  G: 9.1054  DX: 0.4383  DY: 0.5241
Epoch 1/15  Batch 180  G: 7.6075  DX:

KeyboardInterrupt: 

In [ ]:
# %% [markdown]
# Ustvarimo generatorja in diskriminatorja ter inicializiramo uteži.

# %%
G_X2Y = ResnetGenerator(input_nc=3, output_nc=3, n_filters=64, n_blocks=6).to(DEVICE)
G_Y2X = ResnetGenerator(input_nc=3, output_nc=3, n_filters=64, n_blocks=6).to(DEVICE)

D_X = PatchDiscriminator(input_nc=3, n_filters=64).to(DEVICE)  # real vs fake apples
D_Y = PatchDiscriminator(input_nc=3, n_filters=64).to(DEVICE)  # real vs fake oranges


def init_weights(m):
    if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d)):
        nn.init.normal_(m.weight.data, 0.0, 0.02)
        if m.bias is not None:
            nn.init.constant_(m.bias.data, 0.0)
    elif isinstance(m, nn.InstanceNorm2d):
        if m.weight is not None:
            nn.init.normal_(m.weight.data, 1.0, 0.02)
        if m.bias is not None:
            nn.init.constant_(m.bias.data, 0.0)


G_X2Y.apply(init_weights)
G_Y2X.apply(init_weights)
D_X.apply(init_weights)
D_Y.apply(init_weights)

print("Models initialized.")


In [ ]:
# %% [markdown]
# Loss funkcije:
# - GAN: MSE (LSGAN)
# - Cycle: L1
# - Identity: L1
# 
# Optimizatorji: Adam, betas = (0.5, 0.999)

# %%
# GAN loss (LSGAN)
criterion_GAN = nn.MSELoss()
# Cycle & identity loss
criterion_cycle = nn.L1Loss()
criterion_identity = nn.L1Loss()

# Optimizatorji
optimizer_G = optim.Adam(
    itertools.chain(G_X2Y.parameters(), G_Y2X.parameters()),
    lr=LR, betas=(0.5, 0.999)
)
optimizer_D_X = optim.Adam(D_X.parameters(), lr=LR, betas=(0.5, 0.999))
optimizer_D_Y = optim.Adam(D_Y.parameters(), lr=LR, betas=(0.5, 0.999))


def set_requires_grad(nets, requires_grad=False):
    """
    Omogoči/onemogoči gradiente za podane mreže.
    """
    if not isinstance(nets, list):
        nets = [nets]
    for net in nets:
        for p in net.parameters():
            p.requires_grad = requires_grad


def denormalize(tensor):
    """
    Pretvori iz [-1, 1] v [0, 1] za prikaz/shranjevanje.
    """
    out = tensor * 0.5 + 0.5
    return torch.clamp(out, 0.0, 1.0)


In [ ]:
# %% [markdown]
# Glavni training loop:
# - posodabljamo G_X2Y, G_Y2X, D_X, D_Y
# - računamo GAN, cycle in identity loss
# - po vsaki epohi shranimo vizualizacijo (X->Y->X, Y->X->Y)

# %%
G_losses = []
D_X_losses = []
D_Y_losses = []

samples_dir = OUT_DIR / "samples"
samples_dir.mkdir(exist_ok=True, parents=True)

# Vzorec za vizualizacijo (vzemi iz treninga, da sigurno obstaja)
sample_X = next(iter(trainA_loader)).to(DEVICE)  # apple
sample_Y = next(iter(trainB_loader)).to(DEVICE)  # orange

for epoch in range(1, NUM_EPOCHS + 1):
    G_running_loss = 0.0
    D_X_running_loss = 0.0
    D_Y_running_loss = 0.0

    # zipamo loaderja (traja do min dolžine)
    for real_X, real_Y in zip(trainA_loader, trainB_loader):
        real_X = real_X.to(DEVICE)  # jabolka (X)
        real_Y = real_Y.to(DEVICE)  # pomaranče (Y)

        # -----------------------
        # 1) Treniramo generatorja
        # -----------------------
        set_requires_grad([D_X, D_Y], False)
        optimizer_G.zero_grad()

        # Identity loss: G_X2Y(Y) ≈ Y, G_Y2X(X) ≈ X
        id_Y = G_X2Y(real_Y)
        loss_id_Y = criterion_identity(id_Y, real_Y)

        id_X = G_Y2X(real_X)
        loss_id_X = criterion_identity(id_X, real_X)

        loss_identity = (loss_id_X + loss_id_Y) * LAMBDA_ID

        # GAN loss:
        # X -> Y
        fake_Y = G_X2Y(real_X)
        pred_fake_Y = D_Y(fake_Y)
        valid_Y = torch.ones_like(pred_fake_Y, device=DEVICE)
        loss_G_X2Y = criterion_GAN(pred_fake_Y, valid_Y)

        # Y -> X
        fake_X = G_Y2X(real_Y)
        pred_fake_X = D_X(fake_X)
        valid_X = torch.ones_like(pred_fake_X, device=DEVICE)
        loss_G_Y2X = criterion_GAN(pred_fake_X, valid_X)

        loss_GAN = loss_G_X2Y + loss_G_Y2X

        # Cycle loss:
        # X -> Y -> X
        rec_X = G_Y2X(fake_Y)
        loss_cycle_X = criterion_cycle(rec_X, real_X)

        # Y -> X -> Y
        rec_Y = G_X2Y(fake_X)
        loss_cycle_Y = criterion_cycle(rec_Y, real_Y)

        loss_cycle = (loss_cycle_X + loss_cycle_Y) * LAMBDA_CYCLE

        # Skupni loss za generatorja
        loss_G = loss_GAN + loss_cycle + loss_identity
        loss_G.backward()
        optimizer_G.step()

        # -----------------------
        # 2) Treniramo D_X (apples)
        # -----------------------
        set_requires_grad(D_X, True)
        optimizer_D_X.zero_grad()

        # Real X
        pred_real_X = D_X(real_X)
        valid = torch.ones_like(pred_real_X, device=DEVICE)
        loss_D_X_real = criterion_GAN(pred_real_X, valid)

        # Fake X
        pred_fake_X = D_X(fake_X.detach())
        fake = torch.zeros_like(pred_fake_X, device=DEVICE)
        loss_D_X_fake = criterion_GAN(pred_fake_X, fake)

        loss_D_X_total = 0.5 * (loss_D_X_real + loss_D_X_fake)
        loss_D_X_total.backward()
        optimizer_D_X.step()

        # -----------------------
        # 3) Treniramo D_Y (oranges)
        # -----------------------
        set_requires_grad(D_Y, True)
        optimizer_D_Y.zero_grad()

        # Real Y
        pred_real_Y = D_Y(real_Y)
        valid = torch.ones_like(pred_real_Y, device=DEVICE)
        loss_D_Y_real = criterion_GAN(pred_real_Y, valid)

        # Fake Y
        pred_fake_Y = D_Y(fake_Y.detach())
        fake = torch.zeros_like(pred_fake_Y, device=DEVICE)
        loss_D_Y_fake = criterion_GAN(pred_fake_Y, fake)

        loss_D_Y_total = 0.5 * (loss_D_Y_real + loss_D_Y_fake)
        loss_D_Y_total.backward()
        optimizer_D_Y.step()

        # Akumulacija izgub
        G_running_loss += loss_G.item()
        D_X_running_loss += loss_D_X_total.item()
        D_Y_running_loss += loss_D_Y_total.item()

    # Povprečne izgube po epohi
    G_epoch_loss = G_running_loss / len(trainA_loader)
    D_X_epoch_loss = D_X_running_loss / len(trainA_loader)
    D_Y_epoch_loss = D_Y_running_loss / len(trainA_loader)

    G_losses.append(G_epoch_loss)
    D_X_losses.append(D_X_epoch_loss)
    D_Y_losses.append(D_Y_epoch_loss)

    print(f"Epoch [{epoch}/{NUM_EPOCHS}]  "
          f"G_loss: {G_epoch_loss:.4f}  "
          f"D_X_loss: {D_X_epoch_loss:.4f}  "
          f"D_Y_loss: {D_Y_epoch_loss:.4f}")

    # -----------------------
    # Vizualizacija po epohi
    # -----------------------
    with torch.no_grad():
        G_X2Y.eval()
        G_Y2X.eval()

        # apple -> orange -> apple
        fake_Y_sample = G_X2Y(sample_X)
        rec_X_sample = G_Y2X(fake_Y_sample)

        # orange -> apple -> orange
        fake_X_sample = G_Y2X(sample_Y)
        rec_Y_sample = G_X2Y(fake_X_sample)

        G_X2Y.train()
        G_Y2X.train()

        # denormalizacija
        sX = denormalize(sample_X.cpu())
        fY = denormalize(fake_Y_sample.cpu())
        rX = denormalize(rec_X_sample.cpu())

        sY = denormalize(sample_Y.cpu())
        fX = denormalize(fake_X_sample.cpu())
        rY = denormalize(rec_Y_sample.cpu())

        # Funkcija za združevanje 3 slik vodoravno
        def concat3(a, b, c):
            return torch.cat([a, b, c], dim=3)

        grid_X = concat3(sX, fY, rX)
        grid_Y = concat3(sY, fX, rY)

        # skupaj X in Y vertikalno
        vis = torch.cat([grid_X, grid_Y], dim=2)  # [B,C,H_total,W_total]

        vis_img = vutils.make_grid(vis, nrow=1)
        vis_img = vis_img.permute(1, 2, 0).numpy()

        epoch_img_path = samples_dir / f"epoch_{epoch:03d}.png"
        plt.imsave(epoch_img_path, vis_img)


In [ ]:
# %% [markdown]
# Grafa:
# - generator loss
# - discriminator losses (D_X, D_Y)

# %%
plt.figure()
plt.plot(range(1, NUM_EPOCHS + 1), G_losses, label="Generator loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Generator loss (CycleGAN)")
plt.legend()
plt.grid(True)
plt.savefig(OUT_DIR / "generator_loss.png", dpi=150)

plt.figure()
plt.plot(range(1, NUM_EPOCHS + 1), D_X_losses, label="D_X loss")
plt.plot(range(1, NUM_EPOCHS + 1), D_Y_losses, label="D_Y loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Discriminator losses (D_X, D_Y)")
plt.legend()
plt.grid(True)
plt.savefig(OUT_DIR / "discriminator_losses.png", dpi=150)

plt.show()


In [ ]:
# %% [markdown]
# Iz shranjenih epoch_XXX.png naredimo animacijo traininga (GIF).

# %%
frames = []
for epoch in range(1, NUM_EPOCHS + 1):
    img_path = samples_dir / f"epoch_{epoch:03d}.png"
    if img_path.exists():
        frames.append(imageio.imread(img_path))

gif_path = OUT_DIR / "training_progress.gif"
if frames:
    imageio.mimsave(gif_path, frames, fps=2)
    print("Saved GIF to:", gif_path)
else:
    print("No frames found for GIF.")
